## Understanding Retrieval-Augmented Generation (RAG) & Use of Vector Store
*by Ong Chin Ann, edited by Bernice Koh*

#### Objectives
- To understand the concept of Retrival Augmented Generation (RAG)
- To understand the usage of text-embedding model for vectorization
- To familarized with LangChain RAG-related components (Document Loader, Text Splitter, retrievers etc...)
- To develop a custom chatbot with RAG approach


If you find understanding RAG difficult, maybe you can look through the following resource after you have completed the exercise/walkthrough. <br/>
Link: https://www.datacamp.com/blog/what-is-retrieval-augmented-generation-rag


#### RAG Processes
* Step 1: Loading and Chunking Data
* Step 2: Construct Vector Store / Database
* Step 3: Perform Similarity Search based on user Query
* Step 4: Generate response from LLM based on document retrieved from the Vector Database and user query



#### Prerequisites: Set Up

In [1]:
# install all required packages
%pip install -r requirements.txt

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   ------------------- -------------------- 2.6/5.3 MB 16.9 MB/s eta 0:00:01
   --------------------------------- ------ 4.5/5.3 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 5.3/5.3 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   --- ------------------------------------ 1.3/13.8 MB 8.4 MB/s eta 0:00:02
   --------- ------------------------------ 3.4/13.8 MB 9.1 MB/s eta 0:00:02
   --------------- ------------------------ 5.2/13.8 MB 8.6 MB/s eta 0:00:01
   --------------------- ------------------ 7.6/13.8 MB 9.2 MB/s eta 

  You can safely remove it manually.


##### Loading required libraries

Link: 
- PDF Loader: https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
- DOCX Loader: https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/
- FAISS: https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/
- AzureOpenAIEmbeddings: https://python.langchain.com/v0.1/docs/integrations/text_embedding/azureopenai/

In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader

# Import .env variables
load_dotenv(override=True)

True

In [ ]:
# Optional: use rich for pretty printing
from rich import print
from rich.pretty import pprint

#### Step 1a: Document Loader
The following code shows how to load document (docx & pdf) using Docx2txtLoader & PyPDFLoader classes from LangChain

Link: 
- PDF Loader: https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
- DOCX Loader: https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/

In [ ]:
## Loading Docx Documents
faq_loader = Docx2txtLoader("resources/docs/FAQs about the Course.docx")
faq = faq_loader.load()

print("Number of Docs: ", len(faq))

print(faq)

In [ ]:
## Loading PDF Documents
info_loader = PyPDFLoader("resources/docs/SC1015_BasicInformation.pdf")
courseinfo = info_loader.load()

print("Number of Docs: ", len(courseinfo))
print(courseinfo)

#### Step 1b: Document Chunking & Split
When sending a context for LLM for reasoning/inference, its best not to dump in the entire document as this will cause more tokens to be consumed and sometimes can be slow or inaccurate. Hence, there is a need to split the document into chunks which will be stored/index using Vector Store/Databases. 

When parsing the context, we only need to find relevant chunks from the vector store/DB and send over to LLM. This will make the response much more accurate and faster. 

Link: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/character_text_splitter/

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# Define chunk size and chunk overlap
chunk_size = 1000
chunk_overlap = 400

# Define text splitter
text_spliter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Split documents into chunks
faq_chunks = text_spliter.split_documents(faq)
courseinfo_chunks = text_spliter.split_documents(courseinfo)

# Get length of chunks
print("Number of FAQ Chunks: ", len(faq_chunks))
print("Number of Course Info Chunks: ", len(courseinfo_chunks))

In [ ]:
# Preview of chunks
print("Chunk 1 -----------------------")
print(faq_chunks[0])
print("")
print("Chunk 2 -----------------------")
print(faq_chunks[1])

##### Combining both documents' chunks into single collections

In [ ]:
allchunks = []

allchunks.extend(courseinfo_chunks)
allchunks.extend(faq_chunks)

print("Number of Chunks for Course Info: ", len(courseinfo_chunks))
print("Number of Chunks for FAQ: ", len(faq_chunks))
print("Number of Chunks Combined Chunk for both documents: ", len(allchunks))

#### Step 2: Vector Stores, Databases, & Indexing

Now that the chunks from both documents are ready, we will now create an index and store them into a database called Vector Store/Vector Database (Knowledge Base). This is a special database which uses text embedding model which translate a bunch of text into vectors. 

This vector will be useful for search or comparison for similar text given. That's is how the relevant chunk of documents is retrieved when we pass in a query so the vector store/DB could return the relevant chunks of document which will later being passed to LLM as a context/knowledge.

In this exercise, we will be using the Facebook AI Similarity Search (FAISS) as vector store while the AzureOpenAI Text Embedding Model will be use for the text to vector conversion process. Note that FAISS is considered a local database, same goes to ChromaDB while Pinecone and AzureAISearch is considered a cloud-based vector store which uses API calls.

- AzureOpenAI Text Embedding Model: https://python.langchain.com/v0.1/docs/integrations/text_embedding/azureopenai/
- Facebook AI Similarity Search (FAISS) :
    - Vector Database
    - Docs: https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html

*__Acknowledgement__: This access key and resources are supported by NTU EdeX Teaching and Learning Grants 2023-2024 (Call 1)  from the Center for Teaching, Learning & Pedagogy (CTLP) for the project titled "AskNarelle". Please do not share and distribute to other parties*

##### Other Vector Stores/Databases
- ChromaDB
    - Docs: https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/
- Pinecone:
    - Docs: https://python.langchain.com/v0.1/docs/integrations/retrievers/self_query/pinecone/
- AzureAISearch :
    - Docs: https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

# Setting up text embeddings
text_embedding =  AzureOpenAIEmbeddings(
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_APIKEY'],
    azure_deployment=os.environ["AZURE_TEXT_EMBEDDING"],
    model='text-embedding-ada-002'
)

# Creating the FAISS vector database
faissDB = FAISS.from_documents(allchunks, text_embedding)
print("Total chunks & index in FAISS vector database: ", faissDB.index.ntotal)


#### Step 3: Perform Similarity Search based on user Query

We have created a vector database by feeding in the combined chunks for both documents. Now, we will generate a retrieval object which will return top 3 most relavent chunks of document based on a query. 

Docs: https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html#langchain_community.vectorstores.faiss.FAISS.as_retriever

In [ ]:
## Create a retriever for similarity search

# Option 1: Configure to return the top k most similar chunks
# retrieval = faissDB.as_retriever(search_kwargs={"k": 3}) 

# Option 2: Configure to retrieve documents that have a relevance score above a certain threshold
retrieval = faissDB.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.5}
)

retrieval

Now, its time to verify the retrieval and try to search relevant chunk of documents based on query given.

In [ ]:
# Define query
query = "What is the course all about?"

# Retrieve context with retriever
result = retrieval.invoke(query)
print("Total chunks returned: ", len(result))
print(result[0])

print("=====================================================================")
print("--- ALL RETRIEVED CONTENT ----")
for r in result:
    print(r.page_content)
    print("  ### [Source Document: ", r.metadata['source'], "]")
    print("\n     ------------------------------------        \n")
print("=====================================================================")




#### Step 4: Generate response from LLM based on document retrieved and query

This is the last portion where will will construct the logic of the chatbot. The procedure and flow will be as follows:
1) Construct a system prompt
2) Get the user's query
3) The retriever will return relevant chunks of documents based on query given
4) Construct the relevant chunks as the context along with user query
5) Pass the relevant chunk and user query to LLM for inference
6) Display the response and append into chatlog along with query

In [ ]:
from time import sleep

import langchain
from langchain.llms import AzureOpenAI                                          ## This object is a connector/wrapper for OpenAI LLM engine
from langchain_openai import AzureChatOpenAI                                    ## This object is a connector/wrapper for ChatOpenAI engine
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage      ## These are the commonly used chat messages


load_dotenv(override=True)

# Initialise LLM
llm = AzureChatOpenAI(
    openai_api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_APIKEY'],
    azure_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'],
    temperature=1
)

In [ ]:
persona = "You are a teaching assistant at for the course SC1015 at NTU."
task ="your task is to answer student query about the data science and ai course."
context = "the context will be provided based on the course information and FAQ along with the user query"
condition = "If user ask any query beyond data science and ai, tell the user you are not an expert of the topic the user is asking and say sorry. If you are unsure about certain query, say sorry and advise the user to contact the instructor at instructor@ntu.edu.sg"
### any other things to add on

## Constructing initial system message
sysmsg = f"{persona} {task} {context} {condition}"


chatlog = [SystemMessage(content=sysmsg)]

##### Define a function to perform the retrieval and consolidation of the chunks for easy processing

In [ ]:
def search_chunks(query):
    search_result = retrieval.invoke(query)
    context = []
    for r in search_result:
        context.append(r.page_content)

    instruction = "Try to understand the user query and answer based on the context given below:\n"
    return SystemMessage(content=f"{instruction}'context':{context}, 'userquery':{query}")

In [ ]:
# Test out function
x = search_chunks("what is the course all about?")
print(x.content)

##### Chatbot Simulation

In [ ]:
# Chatbot simulation
print("==== CHATBOT SIMULATION ====")
print("---- Enter 'exit' to quit")


query = input("Enter your message: ")
usermsg = HumanMessage(content=query)

# Print first system message
print("System Msg:")
print(sysmsg)

while query != "exit":
    print(f"Human : {query}\n")
    
    # Update chatlog
    chatlog.append(usermsg)

    # Retrieve relevant context
    context = search_chunks(query)

    # Print context
    print("System Msg:")
    print(context.content)
    
    # Update chatlog
    templog = chatlog + [context]

    # Feed updated chatlog to LLM
    response = llm.invoke(templog)

    sleep(2)
    print(f"AI    : {response.content}\n")
    chatlog.append(response)

        
    query = input("Enter your message: ")
    usermsg = HumanMessage(content=query)

In [ ]:
# Print chatlog
print(chatlog)

### CONGRATULATIONS!!!!!

You have completed the basic RAG-LLM Based Chatbot development. In general, you have undergo the fundamental process & pipeline of creating a custom chatbot based on knowledge fed into the vector store (some people call this 'fine-tuning' 😂). This is one out of many ways how a custom chatbot can be constructed. It just like an art on how you can creatively design and develop the pipeline of chatbot based on available technolgies.

In this exercise, we only explore try some technologies for Text Embedding Model (AzureOpenAI Text Embedding), Vector Store/Database (FAISS), and a simple way of crafting the prompts to get desire response (prompt engineering). There are more things beyond what we have covered so far awaitng your exploration. Do explore and try out things differently, and if you discover something interesting and effective, do tell me and maybe you can be the facilitator to showcase your experience on the next event/seminar/workshop/bootcamp!!!!

As of now, you have learnt almost everything you need for a local chatbot development. Why dont try to create one for yourself this evening using the technologies and concept covered i.e. Prompt Engineering, LangChain, Streamlit, & RAG for your own custom chatbot development? Then tomorrow maybe we can see if you can deploy the app to Azure to make it live and publicly accessible!!!

#### Something Extra

In case you wish to persist your vector store, you can actually save it locally and load at the later time so you do not need to undergo Step 1-3 anymore. Below is the simple example.

Link: https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/#saving-and-loading



In [ ]:
# save session's vector store locally
faissDB.save_local("db/sc1015")

#  define new text embeddings
new_text_embedding =  AzureOpenAIEmbeddings(
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_APIKEY'],
    azure_deployment=os.environ["AZURE_TEXT_EMBEDDING"],
    model='text-embedding-ada-002'
)

In [ ]:
# load saved vector store 
new_db = FAISS.load_local("db/sc1015", new_text_embedding, allow_dangerous_deserialization=True)

docs = new_db.similarity_search("What is this course all about?", k=5)

for d in docs:
    print(d)
    print("\n")


Extra Readings:
- https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview